<a href="https://colab.research.google.com/github/ryderwishart/biblical-machine-learning/blob/main/wip/fine_tune_alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install transformers sentencepiece datasets accelerate bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import torch

# Dataset loading and preprocessing

In [3]:
dataset = load_dataset("tatsu-lab/alpaca")

# Split the dataset into train, test, and eval sets
split_dataset = dataset['train'].train_test_split(test_size=0.15)

# Split the dataset into train, test, and eval sets
split_dataset['test'], split_dataset['eval'] = split_dataset['test'].train_test_split(test_size=0.15).values()


# Print the sizes of the datasets
print(f"Train set size: {len(split_dataset['train'])}")
print(f"Test set size: {len(split_dataset['test'])}")
print(f"Eval set size: {len(split_dataset['eval'])}")

  0%|          | 0/1 [00:00<?, ?it/s]

Train set size: 44201
Test set size: 6630
Eval set size: 1171


In [4]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 44201
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 6630
    })
    eval: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 1171
    })
})

# Choose model and tokenizer

In [5]:
from transformers import LlamaTokenizer, LlamaForCausalLM

tokenizer = LlamaTokenizer.from_pretrained("chainyo/alpaca-lora-7b")
model = LlamaForCausalLM.from_pretrained(
    "chainyo/alpaca-lora-7b",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce

Loading checkpoint shards:   0%|          | 0/39 [00:00<?, ?it/s]

In [7]:
def preprocess_data(dataset):
    tokenized_inputs = []
    tokenized_outputs = []
    
    for example in dataset:
        instruction = example["instruction"]
        input_text = example["input"]
        output_text = example["output"]
        
        if input_text:
            input_string = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:"""
        else:
            input_string = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""
            
        tokenized_input = tokenizer.encode(input_string, return_tensors="pt")
        tokenized_output = tokenizer.encode(output_text, return_tensors="pt")

        tokenized_inputs.append(tokenized_input)
        tokenized_outputs.append(tokenized_output)
        
    return tokenized_inputs, tokenized_outputs

train_inputs, train_outputs = preprocess_data(split_dataset['train'])
val_inputs, val_outputs = preprocess_data(split_dataset['test'])


In [9]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

train_dataset = CustomDataset(train_inputs, train_outputs)
val_dataset = CustomDataset(val_inputs, val_outputs)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 39>:39                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:1662 in train                     │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:1899 in _inner_training_loop      │
│                                                                                                  │
│   1896 │   │   │   │   rng_to_sync = True                                                        │
│   1897 │   │   │                                                                                 │
│   1898 │   │   │   step = -1                                                                     │
│ ❱ 1899 │   │   │   for step, inputs in enumerate(epoch_iterator):                                │
│   1900 │   │   │   │   total_batched_samples += 1                                                │
│   1901 │   │   │   │   if rng_to_sync:                                                           │
│   1902 │   │   │   │   │   self._load_rng_state(resume_from_checkpoint)                          │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:634 in __next__            │
│                                                                                                  │
│    631 │   │   │   if self._sampler_iter is None:                                                │
│    632 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    633 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  634 │   │   │   data = self._next_data()                                                      │
│    635 │   │   │   self._num_yielded += 1                                                        │
│    636 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    637 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:678 in _next_data          │
│                                                                                                  │
│    675 │                                                                                         │
│    676 │   def _next_data(self):                                                                 │
│    677 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  678 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    679 │   │   if self._pin_memory:                        

# Evaluation

Goal: use an 'eval' split to compare the answer in the dataset with the answer generated by our now fine-tuned model.

Ryder will help come up with possible answer-quality scoring mechanisms, likely along different dimensions.

Initially, we can just do a cosine distance between the embedding of the generated answer and the embedding of the answer we have in the dataset (which, for now, we will just assume is the 'correct' answer).